# Example: Optimal adversaries for convolutional MNIST model

This notebook gives an example where OMLT is used to find adversarial examples for a trained convolutional neural network. We follow the below steps:<br>
1.) A convolutional neural network (CNN) with ReLU activation functions is trained to classify images from the MNIST dataset <br>
2.) OMLT is used to generate a mixed-integer encoding of the trained CNN using the big-M formulation <br>
3.) The model is optimized to find the maximum classification error (defined by an "adversarial" label) over a small input region <br>



## Library Setup
This notebook assumes you have a working PyTorch environment to train the neural network for classification. The neural network is then formulated in Pyomo using OMLT which therefore requires working Pyomo and OMLT installations.

The required Python libraries used this notebook are as follows: <br>
- `numpy`: used for manipulate input data <br>
- `torch`: the machine learning language we use to train our neural network
- `torchvision`: a package containing the MNIST dataset
- `pyomo`: the algebraic modeling language for Python, it is used to define the optimization model passed to the solver
- `onnx`: used to express trained neural network models
- `omlt`: the package this notebook demonstates. OMLT can formulate machine learning models (such as neural networks) within Pyomo

**NOTE:** This notebook also assumes you have a working MIP solver executable (e.g., CBC, Gurobi) to solve optimization problems in Pyomo. The open-source solver CBC is called by default.

In [1]:
# Import requisite packages
# data manipulation
import tempfile

import numpy as np

# pyomo for optimization
import pyomo.environ as pyo

# pytorch for training neural network
import torch
import torch.onnx
from torch import nn, optim
from torch.optim.lr_scheduler import StepLR
from torchvision import datasets, transforms

# omlt for interfacing our neural network with pyomo
from omlt import OmltBlock
from omlt.io.onnx import (
    load_onnx_neural_network_with_bounds,
    write_onnx_model_with_bounds,
)
from omlt.neuralnet import FullSpaceNNFormulation

## Import the Data and Train a Neural Network

We begin by loading the MNIST dataset as `DataLoader` objects with pre-set training and testing batch sizes:

In [2]:
# set training and test batch sizes
train_kwargs = {"batch_size": 64}
test_kwargs = {"batch_size": 1000}

# build DataLoaders for training and test sets
dataset1 = datasets.MNIST(
    "../data", train=True, download=True, transform=transforms.ToTensor()
)
dataset2 = datasets.MNIST("../data", train=False, transform=transforms.ToTensor())
train_loader = torch.utils.data.DataLoader(dataset1, **train_kwargs, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset2, **test_kwargs)

Next, we define the structure of the convolutional neural network model:

In [3]:
hidden_size = 10


class Net(nn.Module):
    # define layers of neural network
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 2, (4, 4), (2, 2), 0)
        self.conv2 = nn.Conv2d(2, 2, (4, 4), (2, 2), 0)
        self.hidden1 = nn.Linear(5 * 5 * 2, hidden_size)
        self.output = nn.Linear(hidden_size, 10)
        self.relu = nn.ReLU()
        self.softmax = nn.LogSoftmax(dim=1)

    # define forward pass of neural network
    def forward(self, x):
        self.x1 = self.conv1(x)
        self.x2 = self.relu(self.x1)
        self.x3 = self.conv2(self.x2)
        self.x4 = self.relu(self.x3)
        self.x5 = self.hidden1(self.x4.view((-1, 5 * 5 * 2)))
        self.x6 = self.relu(self.x5)
        self.x7 = self.output(self.x6)
        return self.softmax(self.x7)

We next define simple functions for training and testing the neural network:

In [4]:
# training function computes loss and its gradient on batch, and prints status after every 200 batches
def train(model, train_loader, optimizer, epoch):
    model.train()
    criterion = nn.NLLLoss()
    for batch_idx, (data, target) in enumerate(train_loader):
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 200 == 0:
            print(
                f"Train Epoch: {epoch} [{batch_idx * len(data)}/{len(train_loader.dataset)} ({100.0 * batch_idx / len(train_loader):.0f}%)]\tLoss: {loss.item():.6f}"
            )


# testing function computes loss and prints overall model accuracy on test set
def test(model, test_loader):
    model.eval()
    criterion = nn.NLLLoss(reduction="sum")
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            output = model(data)
            test_loss += criterion(output, target).item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()
    test_loss /= len(test_loader.dataset)
    print(
        f"\nTest set: Average loss: {test_loss:.4f}, Accuracy: {correct}/{len(test_loader.dataset)} ({100.0 * correct / len(test_loader.dataset):.0f}%)\n"
    )

Finally, we train the neural network on the dataset.
Training here is performed using the `Adadelta` optimizer for five epochs.

In [5]:
# define model and optimizer
model = Net()
optimizer = optim.Adadelta(model.parameters(), lr=1)
scheduler = StepLR(optimizer, step_size=1, gamma=0.7)

# train CNN model for five epochs
for epoch in range(5):
    train(model, train_loader, optimizer, epoch)
    test(model, test_loader)
    scheduler.step()

Train Epoch: 0 [0/60000 (0%)]	Loss: 2.301499
Train Epoch: 0 [12800/60000 (21%)]	Loss: 2.308466
Train Epoch: 0 [25600/60000 (43%)]	Loss: 2.304558
Train Epoch: 0 [38400/60000 (64%)]	Loss: 2.299439
Train Epoch: 0 [51200/60000 (85%)]	Loss: 2.298130

Test set: Average loss: 2.3012, Accuracy: 1135/10000 (11%)

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.313004
Train Epoch: 1 [12800/60000 (21%)]	Loss: 2.292814
Train Epoch: 1 [25600/60000 (43%)]	Loss: 2.301801
Train Epoch: 1 [38400/60000 (64%)]	Loss: 2.296501
Train Epoch: 1 [51200/60000 (85%)]	Loss: 2.301212

Test set: Average loss: 2.3011, Accuracy: 1135/10000 (11%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 2.291284
Train Epoch: 2 [12800/60000 (21%)]	Loss: 2.307529
Train Epoch: 2 [25600/60000 (43%)]	Loss: 2.297145
Train Epoch: 2 [38400/60000 (64%)]	Loss: 2.307123
Train Epoch: 2 [51200/60000 (85%)]	Loss: 2.305712

Test set: Average loss: 2.3011, Accuracy: 1135/10000 (11%)

Train Epoch: 3 [0/60000 (0%)]	Loss: 2.300129
Train Epoch: 3 [12800/60000 (21%)]	Lo

## Build a MIP Formulation for the Trained Convolutional Neural Network

We are now ready to use OMLT to formulate the trained model within a Pyomo optimization model. We use a  binary variable representation to represent the ReLU activation logic which is created automatically using the `FullSpaceNNFormulation` object (one could also simply use the `ReluBigFormulation` object).

First, we define a neural network without the final `LogSoftmax` activation. Although this activation helps greatly in training the neural network model, it is not trivial to encode in the optimization model. The ranking of the output labels remains the same without the activation, so it can be omitted when finding optimal adversaries. 

In [6]:
class NoSoftmaxNet(nn.Module):
    # define layers of neural network
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 2, (4, 4), (2, 2), 0)
        self.conv2 = nn.Conv2d(2, 2, (4, 4), (2, 2), 0)
        self.hidden1 = nn.Linear(5 * 5 * 2, hidden_size)
        self.output = nn.Linear(hidden_size, 10)
        self.relu = nn.ReLU()

    # define forward pass of neural network
    def forward(self, x):
        x1 = self.conv1(x)
        x2 = self.relu(x1)
        x3 = self.conv2(x2)
        x4 = self.relu(x3)
        x5 = self.hidden1(x4.view((-1, 5 * 5 * 2)))
        x6 = self.relu(x5)
        return self.output(x6)


# create neural network without LogSoftmax and load parameters from existing model
model2 = NoSoftmaxNet()
model2.load_state_dict(model.state_dict())

<All keys matched successfully>

Next, we define an instance of the optimal adversary problem. We formulate the optimization problem as: <br>

$
\begin{align*} 
& \max_x \ y_k - y_j \\
& s.t. y_k = N_k(x) \\ 
&\quad |x - \bar{x}|_\infty \leq 0.05
\end{align*}
$

where $\bar{x}$ corresponds to an image in the test dataset with true label `j`, and $N_k(x)$ is the value of the CNN output corresponding to adversarial label `k` given input `x`. PyTorch needs to trace the model execution to export it to ONNX, so we also define a dummy input tensor `x_temp`.

In [7]:
# load image and true label from test set with index 'problem_index'
problem_index = 0
image = dataset2[problem_index][0].detach().numpy()
label = dataset2[problem_index][1]

# define input region defined by infinity norm
epsilon_infty = 1e-3
lb = np.maximum(0, image - epsilon_infty)
ub = np.minimum(1, image + epsilon_infty)

# save input bounds as dictionary, note that the first index 0 corresponds to the single-channel input
input_bounds = {}
for i in range(28):
    for j in range(28):
        input_bounds[(0, i, j)] = (float(lb[0][i, j]), float(ub[0][i, j]))

# define dummy input tensor
x = dataset2[problem_index][0].view(-1, 1, 28, 28)

We can now export the PyTorch model as an ONNX model and use `load_onnx_neural_network_with_bounds` to load it into OMLT.

In [8]:
with tempfile.NamedTemporaryFile(suffix=".onnx", delete=False) as f:
    # export neural network to ONNX
    torch.onnx.export(
        model2,
        x,
        f.name,
        input_names=["input"],
        output_names=["output"],
        dynamic_axes={"input": {0: "batch_size"}, "output": {0: "batch_size"}},
    )
    # write ONNX model and its bounds using OMLT
    write_onnx_model_with_bounds(f.name, None, input_bounds)
    # load the network definition from the ONNX model
    network_definition = load_onnx_neural_network_with_bounds(f.name)

C:\Users\bammari\AppData\Local\Temp\ipykernel_12024\3796102275.py:3: UserWarning: # 'dynamic_axes' is not recommended when dynamo=True, and may lead to 'torch._dynamo.exc.UserError: Constraints violated.' Supply the 'dynamic_shapes' argument instead if export is unsuccessful.
  torch.onnx.export(


[torch.onnx] Obtain model graph for `NoSoftmaxNet([...]` with `torch.export.export(..., strict=False)`...
[torch.onnx] Obtain model graph for `NoSoftmaxNet([...]` with `torch.export.export(..., strict=False)`... ✅
[torch.onnx] Run decomposition...
[torch.onnx] Run decomposition... ✅
[torch.onnx] Translate the graph into ONNX...
[torch.onnx] Translate the graph into ONNX... ✅


As a sanity check before creating the optimization model, we can print the properties of the neural network layers from `network_definition`. This allows us to check input/output sizes, as well as activation functions.

In [9]:
for layer_id, layer in enumerate(network_definition.layers):
    print(f"{layer_id}\t{layer}\t{layer.activation}")

0	InputLayer(input_size=[1, 28, 28], output_size=[1, 28, 28])	linear
1	ConvLayer(input_size=[1, 28, 28], output_size=[2, 13, 13], strides=[2, 2], kernel_shape=(4, 4))	relu
2	ConvLayer(input_size=[2, 13, 13], output_size=[2, 5, 5], strides=[2, 2], kernel_shape=(4, 4))	relu
3	DenseLayer(input_size=[1, 50], output_size=[1, 10])	relu
4	DenseLayer(input_size=[1, 10], output_size=[1, 10])	linear


Finally, we can load `network_definition` as a full-space `FullSpaceNNFormulation` object.OMLT doesn't include a formulation for sigmoid, so define it here

In [10]:
formulation = FullSpaceNNFormulation(network_definition)

## Solve Optimal Adversary Problem in Pyomo

We now encode the trained neural network in a Pyomo model from the `FullSpaceNNFormulation` object. 

In [11]:
# create pyomo model
m = pyo.ConcreteModel()

# create an OMLT block for the neural network and build its formulation
m.nn = OmltBlock()
m.nn.build_formulation(formulation)

WARNING (W1002): Setting Var 'nn.IndexedVar[0,7,6]' to a numeric value `0`
outside the bounds (0.3284117877483368, 0.33041176199913025).
    See also https://pyomo.readthedocs.io/en/stable/errors.html#w1002
WARNING (W1002): Setting Var 'nn.IndexedVar[0,7,7]' to a numeric value `0`
outside the bounds (0.724490225315094, 0.7264901995658875).
    See also https://pyomo.readthedocs.io/en/stable/errors.html#w1002
WARNING (W1002): Setting Var 'nn.IndexedVar[0,7,8]' to a numeric value `0`
outside the bounds (0.6225294470787048, 0.6245294213294983).
    See also https://pyomo.readthedocs.io/en/stable/errors.html#w1002
WARNING (W1002): Setting Var 'nn.IndexedVar[0,7,9]' to a numeric value `0`
outside the bounds (0.5911568999290466, 0.5931568741798401).
    See also https://pyomo.readthedocs.io/en/stable/errors.html#w1002
WARNING (W1002): Setting Var 'nn.IndexedVar[0,7,10]' to a numeric value `0`
outside the bounds (0.2342941164970398, 0.23629412055015564).
    See also https://pyomo.readthedocs

Next, we define an adversarial label as the true label plus one (or zero if the true label is nine), as well as the objective function for optimization.

In [12]:
adversary = (label + 1) % 10
m.obj = pyo.Objective(expr=(-(m.nn.outputs[0, adversary] - m.nn.outputs[0, label])))

Finally, we solve the optimal adversary problem using a mixed-integer solver.

In [13]:
solver = pyo.SolverFactory("cbc")
solver.solve(m, tee=True)

Welcome to the CBC MILP Solver 
Version: 2.10.8 
Build Date: May  5 2022 

command line - C:\Users\bammari\cbc_solver\bin\cbc.exe -printingOptions all -import C:\Users\bammari\AppData\Local\Temp\tmppsnw49nu.pyomo.lp -stat=1 -solve -solu C:\Users\bammari\AppData\Local\Temp\tmppsnw49nu.pyomo.soln (default strategy 1)
Option for printingOptions changed from normal to all
Presolve 1264 (-2334) rows, 1696 (-1890) columns and 8142 (-6254) elements
Statistics for presolved model
Original problem has 398 integers (398 of which binary)
Presolved problem has 227 integers (227 of which binary)
==== 1690 zero objective 7 different
1 variables have objective of -0.148939
1690 variables have objective of 0
1 variables have objective of 0.107922
1 variables have objective of 0.175845
1 variables have objective of 0.217738
1 variables have objective of 0.286532
1 variables have objective of 1
==== absolute objective values 7 different
1690 variables have objective of 0
1 variables have objective of 0.

{'Problem': [{'Name': 'unknown', 'Lower bound': 0.05256819, 'Upper bound': 0.05256819, 'Number of objectives': 1, 'Number of constraints': 1264, 'Number of variables': 1696, 'Number of binary variables': 398, 'Number of integer variables': 398, 'Number of nonzeros': 6, 'Sense': 'minimize'}], 'Solver': [{'Status': 'ok', 'User time': -1.0, 'System time': 7.62, 'Wallclock time': 7.62, 'Termination condition': 'optimal', 'Termination message': 'Model was solved to optimality (subject to tolerances), and an optimal solution is available.', 'Statistics': {'Branch and bound': {'Number of bounded subproblems': 141, 'Number of created subproblems': 141}, 'Black box': {'Number of iterations': 18046}}, 'Error rc': 0, 'Time': 7.630472898483276}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}